In [1]:
import librosa
import numpy as np
import pandas as pd
import scipy

In [15]:
df = pd.read_csv("../datasets/fma/fma_small_extracted_features.csv")
df.head()

,chroma_cens_00_max,chroma_cens_00_mean,chroma_cens_00_median,chroma_cens_00_min,chroma_cens_00_std,chroma_cens_01_max,chroma_cens_01_mean,chroma_cens_01_median,chroma_cens_01_min,chroma_cens_01_std,...,zcr_00_max,zcr_00_mean,zcr_00_median,zcr_00_min,zcr_00_std,onset_num,beats,tempo,dtempo_changes,Genre
0,0.491591,0.312343,0.301532,0.053283,0.082667,0.395521,0.207771,0.219119,0.006447,0.068787,...,0.224609,0.050325,0.047363,0.005371,0.030025,130,64,129.199219,1,Electronic
1,0.317885,0.074991,0.056387,0.000000,0.070165,0.484911,0.151896,0.144934,0.000000,0.105131,...,0.115234,0.040768,0.037598,0.004395,0.023693,157,48,95.703125,6,Electronic
2,0.451637,0.327583,0.316101,0.225835,0.066120,0.498907,0.346994,0.343436,0.213543,0.074896,...,0.250977,0.091704,0.086914,0.014160,0.043510,131,44,89.102909,1,Electronic
3,0.581031,0.286764,0.244890,0.155665,0.087492,0.427717,0.254131,0.241912,0.215387,0.038424,...,0.353516,0.123570,0.102051,0.015625,0.073909,152,36,71.777344,10,Electronic
4,0.462953,0.262544,0.244547,0.175017,0.061632,0.355859,0.244166,0.244356,0.132059,0.025688,...,0.343750,0.122689,0.099121,0.005859,0.078187,119,50,99.384014,1,Electronic


In [4]:
def columns():
    feature_sizes = dict(chroma_stft=12, chroma_cqt=12, chroma_cens=12,
                         mfcc=12, rms=1, spectral_centroid=1, spectral_bandwidth=1, 
                         spectral_contrast=7, spectral_flatness=1, spectral_rolloff=1,
                         poly_features=3, tonnetz=6, zcr=1, dtempo=1,
                         onset_strength=1, tempogram_ratio=13, plp=1)
    single_features = ['onset_num', 'beats', 'tempo', 'dtempo_changes']
    moments = ('mean', 'std', 'median', 'min', 'max')

    columns = []
    for name, size in feature_sizes.items():
        for moment in moments:
            it = (f"{name}_{i:02d}_{moment}" for i in range(size))
            columns.extend(it)
    # columns.extend(single_features)
    columns = np.sort(np.array(columns))
    columns = np.append(columns, single_features)
    columns = np.append(columns, 'Genre')
    return columns

In [5]:
len(columns())

435

In [21]:
def count_value_changes(arr):
    changes = 0
    for i in range(1, len(arr)):
        if arr[i] != arr[i - 1]:
            changes += 1
    return changes


def calculate_features_for_single_record(file_path):
    y, sr = librosa.load(file_path)
    
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr, n_chroma=12)  #
    chroma_cqt = librosa.feature.chroma_cqt(y=y, sr=sr, n_chroma=12)    #
    chroma_cens = librosa.feature.chroma_cens(y=y, sr=sr)               #
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=12)                  #
    rms = librosa.feature.rms(y=y)                                      #

    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)   #
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr) #
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)   #
    spectral_flatness = librosa.feature.spectral_flatness(y=y)          #
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)     #

    poly_features = librosa.feature.poly_features(y=y, sr=sr, order=2)  #
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr) #
    zcr = librosa.feature.zero_crossing_rate(y)                         #

    onset_env = librosa.onset.onset_strength(y=y, sr=sr)                #
    plp = librosa.beat.plp(onset_envelope=onset_env, sr=sr)             #
    
    dtempo = librosa.feature.tempo(onset_envelope=onset_env, sr=sr, aggregate=None) #
    tempogram_ratio = librosa.feature.tempogram_ratio(tg=librosa.feature.tempogram(y=y, sr=sr), sr=sr) #
    
    # Single features
    dtempo_changes = count_value_changes(dtempo)
    tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
    onset_num = len(librosa.onset.onset_detect(onset_envelope=onset_env, sr=sr))

    moments = ['mean', 'std', 'median', 'min', 'max']

    def aggregate_feature(feature):
        return [np.max(feature), np.mean(feature), np.median(feature), np.min(feature), np.std(feature)]
    
    features = []

    for f in [chroma_cens, chroma_cqt, chroma_stft, dtempo, mfcc, onset_env, plp, poly_features, rms, spectral_bandwidth,
              spectral_centroid, spectral_contrast, spectral_flatness, spectral_rolloff, tempogram_ratio, 
              tonnetz, zcr]:
        if f.ndim == 1:
            features.extend(aggregate_feature(f))
        else:
            features.extend(np.hstack([aggregate_feature(f[i]) for i in range(f.shape[0])]))

    features.append(onset_num)
    features.append(len(beats))
    features.append(tempo[0])
    features.append(dtempo_changes)

    genre = file_path.split('/')[-1].split("\\")[0]
    features.append(genre)

    return features

In [7]:
res = calculate_features_for_single_record('../datasets/fma/fma_small/Hip-Hop/000002.mp3')

In [8]:
len(res)

435

# Creating a features `.csv` file

In [24]:
from tqdm import tqdm
import os

df_features = pd.DataFrame(columns=columns())
df_features

,chroma_cens_00_max,chroma_cens_00_mean,chroma_cens_00_median,chroma_cens_00_min,chroma_cens_00_std,chroma_cens_01_max,chroma_cens_01_mean,chroma_cens_01_median,chroma_cens_01_min,chroma_cens_01_std,...,zcr_00_max,zcr_00_mean,zcr_00_median,zcr_00_min,zcr_00_std,onset_num,beats,tempo,dtempo_changes,Genre


In [25]:
rootdir = '../datasets/fma/fma_small/'
all_features = []

total_files = sum([len(files) for r, d, files in os.walk(rootdir) if any(f.endswith('.mp3') for f in files)])

with tqdm(total=total_files, desc="Processing files") as pbar:
    for subdir, dirs, _ in os.walk(rootdir):
        for folder in dirs:
            folder_path = os.path.join(subdir, folder)
            for _, _, files in os.walk(folder_path):
                for file in files:
                    if file.endswith('.mp3'):
                        path = os.path.join(folder_path, file)
                        try:
                            features = calculate_features_for_single_record(path)
                            # df_features.loc[len(df_features)] = features
                            all_features.append(features)
                            # print(df_features["Genre"])

                        except Exception as e:
                            print(f"Error processing {path}: {e}.")

                        pbar.update(1)

df_features = pd.DataFrame(all_features, columns=columns())

df_features.to_csv('extracted_features.csv', index=False)

Processing files:   0%|          | 11/7994 [00:24<4:50:42,  2.18s/it]


KeyboardInterrupt: 

In [114]:
# df_features.loc[len(df_features)] = res

In [15]:
df_features.head()

,chroma_cens_00_max,chroma_cens_00_mean,chroma_cens_00_median,chroma_cens_00_min,chroma_cens_00_std,chroma_cens_01_max,chroma_cens_01_mean,chroma_cens_01_median,chroma_cens_01_min,chroma_cens_01_std,...,zcr_00_max,zcr_00_mean,zcr_00_median,zcr_00_min,zcr_00_std,onset_num,beats,tempo,dtempo_changes,Genre
0,0.188383,0.083866,0.078871,0.000000,0.043706,0.250713,0.126529,0.136529,0.000000,0.071916,...,0.097168,0.015582,0.013672,0.006836,0.009015,211,97,198.768029,11,Electronic\001482.mp3
1,0.514906,0.361672,0.351852,0.235476,0.066464,0.300494,0.139820,0.136460,0.000000,0.076768,...,0.569336,0.182679,0.164062,0.044922,0.073516,130,42,89.102909,7,Electronic\003573.mp3
2,0.329947,0.193748,0.202888,0.017711,0.060287,0.445039,0.251467,0.241952,0.087461,0.058671,...,0.147949,0.056129,0.055176,0.016602,0.019015,123,47,95.703125,7,Electronic\004519.mp3
3,0.479490,0.273800,0.263448,0.025881,0.091197,0.607575,0.284696,0.274103,0.022886,0.134396,...,0.230957,0.064415,0.058594,0.009766,0.032143,156,76,151.999081,15,Electronic\004520.mp3
4,0.491591,0.312343,0.301532,0.053283,0.082667,0.395521,0.207771,0.219119,0.006447,0.068787,...,0.224609,0.050325,0.047363,0.005371,0.030025,130,64,129.199219,1,Electronic\004521.mp3


# Parallel processing 

In [1]:
import concurrent.futures
import librosa
import pandas as pd
import numpy as np
import scipy
import os
from tqdm import tqdm

In [2]:
def columns() -> np.ndarray:
    feature_sizes = dict(chroma_stft=12, chroma_cqt=12, chroma_cens=12,
                         harmonic_separation=12, percussive_separation=12,
                         tempogram_ratio=13, mfcc=12, spectral_contrast=7, tonnetz=6,
                         poly_features=3, spectral_centroid=1, spectral_bandwidth=1, 
                         spectral_flatness=1, spectral_rolloff=1, rms=1, 
                         zcr=1, onset_strength=1, plp=1, spectral_entropy=1, 
                         autocorelation=1, pitch_features=1, tempo_variability=1, 
                         spectral_decrease=1, dtempo=1)

    single_features = ['tempo', 'beat_count', 'dtempo_changes', 'onset_count', 
                       'low_energy_rate', 'harmonic_to_noise_rate', 'dynamic_range', 
                       'swing_ratio', 'syncopation', 'roughness', 'warmth']
    moments = ('kurtosis', 'max', 'mean', 'median', 'min', 'skew', 'std', 'sum')

    columns = []
    for name, size in feature_sizes.items():
        for moment in moments:
            it = (f"{name}_{i:02d}_{moment}" for i in range(size))
            columns.extend(it)

    columns = np.sort(np.array(columns))
    columns = np.append(columns, single_features)
    columns = np.append(columns, 'Genre')
    return columns

In [6]:
def calculate_features_for_single_record(file_path:str) -> list:
    y, sr = librosa.load(file_path)

    # Chroma features
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr, n_chroma=12)
    chroma_cqt = librosa.feature.chroma_cqt(y=y, sr=sr, n_chroma=12)
    chroma_cens = librosa.feature.chroma_cens(y=y, sr=sr)

    # MFCC, harmonic, Percusive
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=12)
    harmonic, percussive = librosa.effects.hpss(y)
    harmonic_separation = librosa.feature.mfcc(y=harmonic, sr=sr, n_mfcc=12)
    percussive_separation = librosa.feature.mfcc(y=percussive, sr=sr, n_mfcc=12)

    # Tempogram
    tempogram = librosa.feature.tempogram(y=y, sr=sr)
    tempogram_ratio = librosa.feature.tempogram_ratio(tg=tempogram, sr=sr)

    # Tonnetz
    tonnetz = librosa.feature.tonnetz(y=harmonic, sr=sr)

    # Poly features
    poly_features = librosa.feature.poly_features(y=y, sr=sr, order=2)

    # Spectral
    def calculate_spectral_entropy(y, sr):
        psd = np.abs(librosa.stft(y)) ** 2
        psd_sum = np.sum(psd)
        if psd_sum == 0:
            psd_norm = psd
        else:
            psd_norm = psd / (psd_sum + 1e-10)
        entropy = -np.sum(psd_norm * np.log2(psd_norm + 1e-10), axis=0)
        return entropy
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    spectral_flatness = librosa.feature.spectral_flatness(y=y)
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    spectral_entropy = calculate_spectral_entropy(y, sr)

    # RMS
    rms = librosa.feature.rms(y=y)
    
    # ZCR
    zcr = librosa.feature.zero_crossing_rate(y)
    
    # Onset strength
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    # plp
    plp = librosa.beat.plp(onset_envelope=onset_env, sr=sr)

    # Autocorrelation
    autocorrelation = librosa.autocorrelate(y)

    # Pitch features
    pitches, magnitudes = librosa.core.piptrack(y=y, sr=sr)
    pitch_features = pitches[pitches > 0]

    # Tempo variability
    tempo_variability = librosa.feature.rhythm.tempo(onset_envelope=onset_env, sr=sr, aggregate=None)

    # Spectral decrease
    def calculate_spectral_decrease(y, sr):
        S = np.abs(librosa.stft(y))
        decrease = np.mean(np.diff(S, axis=0), axis=1)
        return decrease
    spectral_decrease = calculate_spectral_decrease(y, sr)

    # Dynamic tempo
    dtempo = librosa.feature.tempo(onset_envelope=onset_env, sr=sr, aggregate=None)
    

    # Single features
    def count_value_changes(arr:list) -> int:
        changes = 0
        for i in range(1, len(arr)):
            if arr[i] != arr[i - 1]:
                changes += 1
        return changes

    def calculate_swing_ratio(y, sr):
        onset_frames = librosa.onset.onset_detect(y=y, sr=sr)
        onset_times = librosa.frames_to_time(onset_frames, sr=sr)
        swing_ratios = []
        for i in range(1, len(onset_times)-1, 2):
            duration_1 = onset_times[i] - onset_times[i-1]
            duration_2 = onset_times[i+1] - onset_times[i]
            if duration_2 != 0:
                swing_ratio = duration_1 / duration_2
                swing_ratios.append(swing_ratio)
        return np.mean(swing_ratios) if swing_ratios else 0
    
    def calculate_syncopation(beats, sr):
        onset_frames = librosa.onset.onset_detect(y=y, sr=sr)
        onset_times = librosa.frames_to_time(onset_frames, sr=sr)
        beat_times = librosa.frames_to_time(beats, sr=sr)
        syncopation = 0
        for onset in onset_times:
            closest_beat = min(beat_times, key=lambda x: abs(x - onset))
            syncopation += abs(onset - closest_beat)
        return syncopation / len(onset_times) if len(onset_times) else 0
    
    def calculate_roughness(harmonic, sr):
        S = np.abs(librosa.stft(harmonic))
        frequencies = librosa.fft_frequencies(sr=sr)
        magnitudes = np.mean(S, axis=1)
        roughness = 0
        for i in range(len(frequencies) - 1):
            for j in range(i + 1, len(frequencies)):
                if abs(frequencies[i] - frequencies[j]) < 20:
                    roughness += magnitudes[i] * magnitudes[j] / abs(frequencies[i] - frequencies[j])
        return roughness
    
    def calculate_warmth(y, sr):
        mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
        S_db = librosa.power_to_db(mel_spectrogram, ref=np.max)
        low_freq_idx = np.where(librosa.mel_frequencies(n_mels=128, fmax=sr/2) < 200)[0]
        low_freq_mean = np.mean(S_db[low_freq_idx, :])
        overall_mean = np.mean(S_db)
        if overall_mean == 0:
            return 0
        warmth = low_freq_mean / (overall_mean + 1e-10)
        return warmth

    dtempo_changes = count_value_changes(dtempo)
    tempo, beats = librosa.beat.beat_track(onset_envelope=onset_env, sr=sr)
    beat_count = len(beats)
    onset_count = len(librosa.onset.onset_detect(onset_envelope=onset_env, sr=sr))
    low_energy = np.sum(rms < 0.5 * np.mean(rms)) / len(rms)

    def calculate_harmonic_ratio(harmonic, percussive):
        harmonic_sum = np.sum(harmonic)
        percussive_sum = np.sum(percussive)
        denominator = percussive_sum + harmonic_sum
        if denominator == 0:
            return 0
        harmonic_ratio = harmonic_sum / denominator
        return harmonic_ratio
        
    harmonic_ratio = calculate_harmonic_ratio(harmonic, percussive)
    dynamic_range = np.max(rms) - np.min(rms)
    swing_ratio = calculate_swing_ratio(y, sr)
    syncopation = calculate_syncopation(beats, sr)
    roughness = calculate_roughness(harmonic, sr)
    warmth = calculate_warmth(y, sr)

    moments = ['kurtosis', 'max', 'mean', 'median', 'min', 'skew', 'std', 'sum']

    def aggregate_feature(feature):
        if np.allclose(feature, feature[0]):
            return [np.nan, np.max(feature), np.mean(feature), np.median(feature), 
                np.min(feature), np.std(feature), np.nan, sum(feature)]
        else:
            return [scipy.stats.kurtosis(feature), np.max(feature), np.mean(feature), np.median(feature), 
                    np.min(feature), np.std(feature), scipy.stats.skew(feature), sum(feature)]
    
    features = []

    for f in [autocorrelation, chroma_cens, chroma_cqt, chroma_stft, dtempo, harmonic_separation,
              mfcc, onset_env, percussive_separation, pitch_features, plp, poly_features, 
              rms, spectral_bandwidth, spectral_centroid, spectral_contrast, spectral_decrease, 
              spectral_entropy, spectral_flatness, spectral_rolloff, tempo_variability, 
              tempogram_ratio, tonnetz, zcr]:
        if f.ndim == 1:
            features.extend(aggregate_feature(f))
        else:
            features.extend(np.hstack([aggregate_feature(f[i]) for i in range(f.shape[0])]))
    
    single_features = [tempo[0], beat_count, dtempo_changes, onset_count, low_energy, harmonic_ratio,
                       dynamic_range, swing_ratio, syncopation, roughness, warmth]
    features.extend(single_features)

    genre = file_path.split('\\')[-2]
    features.append(genre)

    return features

In [4]:

def process_file(file_path:str)->list:
    try:
        features = calculate_features_for_single_record(file_path)
        return features
    except Exception as e:
        print(f"Error processing {file_path}: {e}.")
        return None


def process_files_in_parallel(rootdir:str) -> list():
    all_features = []

    # Count total files for progress bar
    total_files = sum([len(files) for r, d, files in os.walk(rootdir) if any(f.endswith('.mp3') for f in files)])

    # Using ThreadPoolExecutor to process files in parallel
    with tqdm(total=total_files, desc="Processing files") as pbar, concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        
        for subdir, dirs, _ in os.walk(rootdir):
            for folder in dirs:
                folder_path = os.path.join(subdir, folder)
                for _, _, files in os.walk(folder_path):
                    for file in files:
                        if file.endswith('.mp3'):
                            path = os.path.join(folder_path, file)
                            futures.append(executor.submit(process_file, path))
        
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            if result is not None:
                all_features.append(result)
            pbar.update(1)

    return all_features


def create_features_as_csv(rootdir:str, save_path:str) -> None:
    df_features = pd.DataFrame(process_files_in_parallel(rootdir), columns=columns())
    df_features.to_csv(save_path, index=False)

In [5]:
create_features_as_csv(rootdir=r'..\\..\\datasets\\fma\\fma_small', save_path=r'..\\..\\datasets\\fma\\extracted_features.csv')

Processing files:   0%|          | 0/7994 [00:00<?, ?it/s]

Processing files:  10%|▉         | 775/7994 [15:57<1:52:16,  1.07it/s]c:\Users\szyme\Documents\Praca_Magisterska\Musical-Genres-Classification\.venv\lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
Processing files:  53%|█████▎    | 4235/7994 [1:35:00<31:39,  1.98it/s]  

Error processing ..\\..\\datasets\\fma\\fma_small\Instrumental\107535.mp3: index 0 is out of bounds for axis 0 with size 0.


Processing files: 100%|██████████| 7994/7994 [2:58:34<00:00,  1.34s/it]  
